In [1]:
# import tensorflow as tf
# from keras import backend as K

# num_cores = 4
# CPU = True
# GPU = False

# if GPU:
#     num_GPU = 1
#     num_CPU = 1
# if CPU:
#     num_CPU = 1
#     num_GPU = 0

# config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
#         inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
#         device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
# session = tf.Session(config=config)
# K.set_session(session)

Using TensorFlow backend.


In [1]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import *
import matplotlib.pyplot as plt
from tokenizer import tokenizer as tweet_tokenizer
from keras.utils.np_utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import *
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.optimizers import * 


%matplotlib inline

PATH = '/home/epita/sim/sentiment_analysis/data/'

train_3 = f'{PATH}/data_train_3.csv'
test_3 = f'{PATH}/data_test_3.csv'
train_7 = f'{PATH}/data_train_7.csv'
train_16m_3 = f'{PATH}/training.1600000.processed.noemoticon.csv'

Using TensorFlow backend.


## One hot sample

In [16]:
text = 'The quick brown fox jumped over the lazy dog.'

words = set(text_to_word_sequence(text))
vocab_size = len(words)
print(vocab_size)

result = one_hot(text, round(vocab_size*1.3))
print(result)

8
[6, 2, 7, 1, 4, 4, 6, 6, 7]


## Tweet Tokenizer sample

In [60]:
T = tweet_tokenizer.TweetTokenizer(preserve_case=False, preserve_url=False)

text = 'The quick brown fox jumped over the lazy dog.'

T.tokenize(text)

['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', '.']

## Lemmatizer / Stemmer samples

In [22]:
from nltk.stem import PorterStemmer, WordNetLemmatizer
 
stemmer = PorterStemmer()
lemmatiser = WordNetLemmatizer()
 
print("Stem %s: %s" % ("studying", stemmer.stem("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying")))
print("Lemmatise %s: %s" % ("studying", lemmatiser.lemmatize("studying", pos="v")))

Stem studying: studi
Lemmatise studying: studying
Lemmatise studying: study


In [33]:
from nltk import pos_tag
from nltk.tokenize import word_tokenize
from collections import defaultdict
from nltk.corpus import wordnet as wn


tag_map = defaultdict(lambda: wn.NOUN, {'J': wn.ADJ, 'V': wn.VERB, 'R': wn.ADV})

s = "This is a simple sentence that would be allowing us to try lemmatizing"

tokens = word_tokenize(s)
tokens_pos = pos_tag(tokens)
 
print(tokens_pos)


lems = [lemmatiser.lemmatize(word, tag_map[pos[0]]) for word, pos in tokens_pos]
print(lems)

[('This', 'DT'), ('is', 'VBZ'), ('a', 'DT'), ('simple', 'JJ'), ('sentence', 'NN'), ('that', 'WDT'), ('would', 'MD'), ('be', 'VB'), ('allowing', 'VBG'), ('us', 'PRP'), ('to', 'TO'), ('try', 'VB'), ('lemmatizing', 'VBG')]
['This', 'be', 'a', 'simple', 'sentence', 'that', 'would', 'be', 'allow', 'u', 'to', 'try', 'lemmatizing']


# Serious business

## Preprocess Train_16m_3

In [19]:
tweets_16m = pd.read_csv(train_16m_3, sep=',', encoding='latin-1').drop(columns=['date', 'flag', 'user'])

print(tweets_16m.shape)
tweets_16m.sample(5)

(1600000, 3)


,target,id,text
1136298,4,1976586207,@Doc_Sprocket sorry - i thought i wasn't going...
82391,0,1753038955,horrible headache
883382,4,1686122324,YES! ... finished my assignment! 1 down and 3 ...
1473925,4,2065642051,best #kiss i ever seen http://bit.ly/kdepj
795603,0,2327426618,"thinking she failed her geo exam, and possibly..."


In [20]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets_16m['target'] = tweets_16m['target'].apply(lambda x: {0: 0, 2: 1, 4: 2}[x])  # Neg (0), Neu (2), Pos (4)
tweets_16m['text'] = tweets_16m['text'].apply(lambda x: standardization(x))

In [21]:
display(tweets_16m.sample(5))
tweets_16m.sample(5)['text'].iloc[0]

,target,id,text
174553,0,1964566165,babe u aint read tweet phone fall apart new co...
235416,0,1979777211,not cold go get
901553,2,1694256723,get new frens-frm germany mexico n france havi...
913630,2,1752671998,im not cookie wont comment heh girl sing crush...
832154,2,1557553994,ah sunday morning nice day get realize money a...


'night twitterverse seek ever elusive prince sleep sweet dream u good morning ever apply zzzz'

In [23]:
tweets_16m.to_feather('pickles/train_16m_3.feather')

## Preprocess Train_3

In [2]:
tweets = pd.read_csv(train_3, sep='\t', names=['ID', 'Class', 'Tweet'])

tweets.shape

(50333, 3)

In [3]:
tweets.sample(5)

,ID,Class,Tweet
13666,522735017957814272,neutral,View the program for the new RNA Nanotechnolog...
9474,251159660458024960,neutral,@SouthSideAsylum no back to watching Jim Henso...
28417,635916750328991746,neutral,Kris Bryant homered in the bottom of the 9th. ...
31083,638338687902744576,neutral,Canadians would actually love it if Scott Walk...
47663,260158752907468800,neutral,Did you know that every Monday is Facebook Mon...


In [3]:
from TP_transfer_learning_2018 import *
from TP_transfer_learning_2018.preprocessing import standardization


tweets['Sentiment'] = tweets['Class'].apply(lambda x: {'negative': 0, 'neutral': 1, 'positive': 2}[x])
tweets['Tweet'] = tweets['Tweet'].apply(lambda x: standardization(x))

In [4]:
display(tweets.sample(5))
tweets.sample(5)['Tweet'].iloc[0]

,ID,Class,Tweet,Sentiment
15079,522833226662813696,positive,ben affleck henry cavill city suck know 3rd se...,2
43712,628587233365266432,positive,galaxy note event schedule august 13 seem thou...,2
10248,264075183738998785,positive,.. morning austin ... go bed .. hahha 8: 45pm ...,2
40095,680397873910005760,neutral,let christ king israel come cross may see believe,1
47884,247139201496260608,neutral,jcc demand rop workcharge employee urge chief ...,1


'lumia xl rumor t-mobile may get first dibs official press render cityman ...'

In [22]:
tweets.to_feather('pickles/train_3.feather')

## Training

In [5]:
# train_tweets, sentiments = tweets['Tweet'], tweets['Sentiment']
train_tweets, sentiments = tweets_16m['text'], tweets_16m['target']

all_tweets = train_tweets # + test_tweets
tokenizer = Tokenizer(filters=' ')
tokenizer.fit_on_texts(all_tweets)
word_index = tokenizer.word_index

In [6]:
train_sequences = tokenizer.texts_to_sequences(train_tweets)

sequences = train_sequences # + test_sequences
MAX_SEQUENCE_LENGTH = 0
for elt in sequences:
    if len(elt) > MAX_SEQUENCE_LENGTH:
        MAX_SEQUENCE_LENGTH = len(elt)

MAX_SEQUENCE_LENGTH

32

In [7]:
train_sequences = pad_sequences(train_sequences, MAX_SEQUENCE_LENGTH)
train_sequences.shape

(50333, 32)

In [8]:
googlenews_w2v = KeyedVectors.load_word2vec_format('data/embeddings/GoogleNews-vectors-negative300.bin', binary=True)

In [9]:
targets = to_categorical(sentiments, 3)
nb_words = len(word_index) + 1

EMBEDDING_DIM = 300
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

oov = []  # Out of vocabulary
oov.append((np.random.rand(EMBEDDING_DIM) * 2.0) - 1.0)
oov = oov / np.linalg.norm(oov)

print(oov.shape)

for word, i in word_index.items():
    if word in googlenews_w2v.vocab:
        embedding_matrix[i] = googlenews_w2v.word_vec(word)
    else:
        embedding_matrix[i] = oov
        
print(embedding_matrix.shape)

(1, 300)
(36968, 300)


In [10]:
# Claim memory back from this very large object we don't use anymore
del googlenews_w2v

In [11]:
X_train, X_val, y_train, y_val = train_test_split(train_sequences, targets, test_size=0.3)

print('training set: ' + str(len(X_train)) + ' samples')
print('validation set: ' + str(len(X_val)) + ' samples')

print('x_train:', X_train.shape)
print('y_train:', y_train.shape)

training set: 35233 samples
validation set: 15100 samples
x_train: (35233, 32)
y_train: (35233, 3)


In [12]:
model = Sequential()
model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], 
                    input_length=MAX_SEQUENCE_LENGTH, trainable=False, name='embedding_layer'))
model.add(Dropout(0.3))
# model.add(LSTM(150))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
# model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(150, return_sequences=True)))
model.add(Dropout(0.5))
# model.add(Dense(32, activation='relu'))
# model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-2), metrics=['acc'])
          
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 32, 300)           11090400  
_________________________________________________________________
dropout_1 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 32, 300)           541200    
_________________________________________________________________
dropout_2 (Dropout)          (None, 32, 300)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 9600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 28803     
Total para

In [13]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=5)

Train on 35233 samples, validate on 15100 samples
Epoch 1/5
35200/35233 [============================>.] - ETA: 0s - loss: 0.8946 - acc: 0.5915

KeyboardInterrupt: 

In [25]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=15)

Train on 35233 samples, validate on 15100 samples
Epoch 1/15
35233/35233 [==============================] - 131s 4ms/step - loss: 0.6115 - acc: 0.7292 - val_loss: 0.7785 - val_acc: 0.6544
Epoch 2/15
35233/35233 [==============================] - 131s 4ms/step - loss: 0.5821 - acc: 0.7434 - val_loss: 0.7843 - val_acc: 0.6644
Epoch 3/15
35233/35233 [==============================] - 129s 4ms/step - loss: 0.5598 - acc: 0.7580 - val_loss: 0.8256 - val_acc: 0.6501
Epoch 4/15
35233/35233 [==============================] - 131s 4ms/step - loss: 0.5369 - acc: 0.7709 - val_loss: 0.8124 - val_acc: 0.6609
Epoch 5/15
35233/35233 [==============================] - 132s 4ms/step - loss: 0.5178 - acc: 0.7779 - val_loss: 0.8663 - val_acc: 0.6523
Epoch 6/15
35233/35233 [==============================] - 132s 4ms/step - loss: 0.5016 - acc: 0.7857 - val_loss: 0.8337 - val_acc: 0.6562
Epoch 7/15
35233/35233 [==============================] - 130s 4ms/step - loss: 0.4966 - acc: 0.7890 - val_loss: 0.9111 - 

### Bi-derectional LSTM x2

In [20]:
model.fit(X_train, y_train, batch_size=128, validation_data=(X_val, y_val), epochs=20)

Train on 35233 samples, validate on 15100 samples
Epoch 1/20
35233/35233 [==============================] - 133s 4ms/step - loss: 0.8855 - acc: 0.5888 - val_loss: 0.7555 - val_acc: 0.6511
Epoch 2/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7755 - acc: 0.6376 - val_loss: 0.7467 - val_acc: 0.6560
Epoch 3/20
35233/35233 [==============================] - 128s 4ms/step - loss: 0.7400 - acc: 0.6575 - val_loss: 0.7229 - val_acc: 0.6686
Epoch 4/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.7124 - acc: 0.6727 - val_loss: 0.7285 - val_acc: 0.6725
Epoch 5/20
35233/35233 [==============================] - 129s 4ms/step - loss: 0.6925 - acc: 0.6867 - val_loss: 0.7240 - val_acc: 0.6644
Epoch 6/20
35233/35233 [==============================] - 131s 4ms/step - loss: 0.6774 - acc: 0.6938 - val_loss: 0.7288 - val_acc: 0.6676
Epoch 7/20
35233/35233 [==============================] - 122s 3ms/step - loss: 0.6579 - acc: 0.7035 - val_loss: 0.7342 - 

### Logistic Regression

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(binary=True)

X = cv.fit_transform(tweets['Tweet'])
target = tweets['Sentiment']

In [7]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(X, target)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    lr = LogisticRegression(C=c, solver='saga', multi_class='auto')
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(y_val, lr.predict(X_val))))

Accuracy for C=0.01: 0.6240464081373173
Accuracy for C=0.05: 0.6523363000635728
Accuracy for C=0.25: 0.6594882390336936
Accuracy for C=0.5: 0.6562301335028607
Accuracy for C=1: 0.6511443102352193


#### Best and worst features

In [11]:
feature_to_coef = {word: coef for word, coef in zip(cv.get_feature_names(), lr.coef_[0])}

for best_positive in sorted(feature_to_coef.items(), key=lambda x: x[1])[:5]:
    print (best_positive)
    
print()
for best_negative in sorted(feature_to_coef.items(), key=lambda x: x[1], reverse=True)[:5]:
    print (best_negative)

('gifford', -1.2377102595086205)
('seinfeld', -1.208227717966838)
('bout', -1.121596723479545)
('bless', -1.1170007079307602)
('greatest', -1.0976803765492371)

('worst', 2.4448066170437612)
('sucks', 2.221116192138116)
('stupid', 2.2174777929515206)
('fucked', 2.1713174119575873)
('fuck', 2.127179056507483)


## Learning (pseudo-code)

In [ ]:
Sequential()
Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)
LSTM(32)
Dropout(0.2)
Dense(32, 'relu')
Dropout(0.2)
Dense(3, activation='softmax')

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['acc'])

model.fit(...)

In [ ]:
# transfer:

model.layers.pop()
model.layers.pop()
Dense(150, 'relu')
Dense(64, 'relu')
Dense(7, 'softmax')
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

## Notes
* Télécharger 50 millions de tweets pour le word2vec
* Metric pour le script du prof: pearson
* Utiliser pearson au lieu d'accuracy
* L'année dernière le prof a eu 78% accuracy 

**Output attendu**: produire un fichier avec les mêmes noms de cartégories sur un fichier de test façon kaggle